<a href="https://colab.research.google.com/github/JarekMaleszyk/data-science-project-sandbox/blob/main/power_bi_client_package_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
try:
  from powerbiclient import Report, models
except ImportError:
  !pip install powerbiclient -q
  from powerbiclient import Report, models
finally:
  from powerbiclient.authentication import DeviceCodeLoginAuthentication
  from io import StringIO
  from ipywidgets import interact
  import requests
  import pandas as pd
  import matplotlib

In [12]:
auth = DeviceCodeLoginAuthentication()

Performing device flow authentication. Please follow the instructions below.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BMVREAG36 to authenticate.


KeyboardInterrupt: 

In [ ]:
group_id = "bafb44a9-9fe9-4b67-9b01-23ee2ea116b7"
report_id = "d7bb690b-2e94-47cd-82e0-ba4d7c04f0c7"
report = Report(group_id=group_id, report_id=report_id, auth=auth)

In [ ]:
def loaded_callback(event_details):
    print('Report is loaded')

report.on('loaded', loaded_callback)

In [ ]:
def rendered_callback(event_details):
    print('Report is rendered')

report.on('rendered', rendered_callback)

In [ ]:
# Render report
report

In [ ]:
def get_active_page(self):

    # Get list of pages
    pages = self.get_pages()
    active_page = {}
    for page in pages:

        # Get active page
        if page['isActive'] == True:
            active_page = page
            break
    return active_page

In [ ]:
active_page = get_active_page(report)
active_page_name = active_page['name']

# Get list of visuals on active page
visuals = report.visuals_on_page(active_page_name)

# Get visual whose type is 'clusteredColumnChart'
visual = next(filter(lambda visual: visual['type'] == 'clusteredColumnChart', visuals))
visual_name = visual['name']

In [ ]:

summarized_exported_data = report.export_visual_data(active_page_name, visual_name, rows = 20)

print(summarized_exported_data)

In [ ]:
underlying_exported_data = report.export_visual_data(active_page_name, visual_name, rows = 5, export_data_type = models.ExportDataType.UNDERLYING.value)

print(underlying_exported_data)

In [ ]:
data = StringIO(summarized_exported_data)

df = pd.read_csv(data, sep=",")
df.head()

In [ ]:
df.plot.pie(y="Total Units", labels=df.loc[:,'isVanArsdel'].values.tolist(), figsize=(10,15))

In [ ]:
def filter_report(Region):
    region_filter = {
        '$schema': "http://powerbi.com/product/schema#basic",
        'target': {
            'table': "Geo",
            'column': "Region"
        },
        'operator': "In",
        'values': [Region]
    }
    report.remove_filters()
    report.update_filters([region_filter])

In [ ]:
report

In [ ]:
interact(filter_report, Region=['Central','East', 'West']);

In [ ]:
report.get_filters()

In [ ]:
# Get list of bookmarks
bookmarks_list = report.get_bookmarks()

# Convert the list into a dataframe
bookmarks_dataframe = pd.DataFrame(bookmarks_list)

# Rename columns
bookmarks_dataframe.rename(columns={'name':'Name', 'displayName':'Display Name', 'state':'State'}, inplace=True)
bookmarks_dataframe

In [ ]:
def apply_bookmark(BookmarkDisplayName):

    # Get bookmark name corresponding to the display name provided by the user
    bookmark_name = bookmarks_dataframe[bookmarks_dataframe['Display Name'] == BookmarkDisplayName].Name.values[0]
    report.set_bookmark(bookmark_name)

In [ ]:
report

In [ ]:
interact(apply_bookmark, BookmarkDisplayName=bookmarks_dataframe['Display Name'])